In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import BatchNorm1d
class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=0)
        
        # Bidirectional LSTM
        self.bilstm_1 = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout,
        )

        # Batch normalization
        #self.batchnorm_1 = BatchNorm1d(hidden_dim * 2)

        # Fully connected layers
        self.fc2 = nn.Linear(hidden_dim * 2, output_dim)


    def forward(self, x):
        # Embedding layer
        embedded = self.embedding(x)

        # BiLSTM layer
        lstm_out, _ = self.bilstm_1(embedded)

        # # # Batch normalization
        # lstm_out = lstm_out.permute(0, 2, 1)
        # lstm_out = self.batchnorm_1(lstm_out)
        # lstm_out = lstm_out.permute(0, 2, 1)

        output = self.fc2(lstm_out)
        return F.log_softmax(output, dim=-1)



In [2]:
import pandas as pd
train_path = '../dataset/PIZZA_train.json'
test_path = '../dataset/PIZZA_dev.json'
df = pd.read_json(train_path, lines=True)
dev = pd.read_json(test_path, lines=True)

In [3]:
df.describe()

,train.SRC,train.EXR,train.TOP,train.TOP-DECOUPLED
count,2456446,2456446,2456446,2456446
unique,2456446,694346,2456446,1425035
top,can i have a large bbq pulled pork,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZ...,(ORDER (PIZZAORDER (NUMBER three ) (NOT (TOPPI...
freq,1,1999,1,167


In [4]:
unique_exr = df['train.TOP'].value_counts()[df['train.TOP'].value_counts() == 1].index

# Filter rows where 'train.EXR' is unique
unique_related_dataset = df[df['train.TOP'].isin(unique_exr)].reset_index(drop=True)
unique_related_dataset

,train.SRC,train.EXR,train.TOP,train.TOP-DECOUPLED
0,can i have a large bbq pulled pork,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (...,(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZ...,(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (...
1,large pie with green pepper and with extra pep...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (...,(ORDER (PIZZAORDER (SIZE large ) pie with (TOP...,(ORDER (PIZZAORDER (SIZE large ) (TOPPING gree...
2,i'd like a large vegetarian pizza,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (...,(ORDER i'd like (PIZZAORDER (NUMBER a ) (SIZE ...,(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (...
3,party size stuffed crust pie with american che...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER (PIZZAORDER (SIZE party size ) (STYLE s...,(ORDER (PIZZAORDER (SIZE party size ) (STYLE s...
4,can i have one personal sized artichoke,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PERSONAL_...,(ORDER can i have (PIZZAORDER (NUMBER one ) (S...,(ORDER (PIZZAORDER (NUMBER one ) (SIZE persona...
...,...,...,...,...
2456441,i'd like a pizza with arugula ricotta cheese a...,(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING ARUGUL...,(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza ...,(ORDER (PIZZAORDER (NUMBER a ) (TOPPING arugul...
2456442,i'd like a pizza with yellow peppers fried oni...,(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING YELLOW...,(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza ...,(ORDER (PIZZAORDER (NUMBER a ) (TOPPING yellow...
2456443,i'd like a pizza with olives roasted tomatoes ...,(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING OLIVES...,(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza ...,(ORDER (PIZZAORDER (NUMBER a ) (TOPPING olives...
2456444,i'd like a pizza with mozzarella jalapeno and ...,(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING MOZZAR...,(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza ...,(ORDER (PIZZAORDER (NUMBER a ) (TOPPING mozzar...


In [5]:
#unique_patterns = df['train.EXR'].value_counts()
# # Subset the DataFrame for rows with unique values
# unique_related_dataset = df[df['train.EXR'].isin(unique_patterns.index[:5000])] # reduce to 5000 patterns

# # Describe the resulting dataset
# unique_related_dataset.describe()

In [6]:
# unique_related_dataset =unique_related_dataset.reset_index(drop=True)
# unique_related_dataset

In [7]:
X_train = unique_related_dataset['train.SRC']
y_train = unique_related_dataset['train.TOP']
X_test = dev['dev.SRC']
y_test = dev['dev.TOP']
print(len(unique_related_dataset))
print(X_train[476368])
print(y_train[476368])
print(dev['dev.SRC'][0])

2456446
i want three pies with parmesan cheese and without any sauce
(ORDER i want (PIZZAORDER (NUMBER three ) pies with (TOPPING parmesan cheese ) and without any (NOT (TOPPING sauce ) ) ) )
i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage


In [8]:
import sys
sys.path.append("..")
from utils.data_preprocessing import preprocess_text
from utils.feature_extraction import bag_of_words, tfidf_features, extract_embeddings
X_train = [" ".join(preprocess_text(text)) for text in X_train]
X_test = [" ".join(preprocess_text(text)) for text in X_test]
y_train = [" ".join(preprocess_text(text)) for text in y_train]
y_test = [" ".join(preprocess_text(text)) for text in y_test]

In [9]:
del df
del dev

In [10]:
X_train

['a large bbq pulled pork',
 'large pie with green pepper with extra pepperoni',
 'like a large vegetarian pizza',
 'party size stuffed crust pie with american cheese with mushroom',
 'one personal sized artichoke',
 'pie with banana pepper pepperoni extra low fat cheese',
 'want one regular pizza without any fried onion',
 'want a stuffed crust pizza with american cheese a little bit pepperoni',
 'one party sized high rise dough pizza with american cheese a lot pepperoni',
 'pie with green olive pesto sauce',
 'one pie with banana pepper pepperoni yellow pepper',
 'like one party sized pie with american cheese with pesto sauce',
 'one high rise dough pie with american cheese a lot meatball',
 'a party sized pie without any bean',
 'party sized big meat pizza with american cheese mushroom',
 'like a party sized high rise dough pie with a lot banana pepper pecorino cheese',
 'like a party sized pie with balsamic glaze black olive',
 'party size pie with chicken mozzarella with extra sau

In [11]:
y_train[0]

'ORDER PIZZAORDER NUMBER a SIZE large TOPPING bbq pulled pork'

SRC
party sized high rise dough pie with american cheese with oregano
TOP
party sized high rise dough pie american cheese with oregano


ORDER PIZZAORDER SIZE party sized STYLE high rise dough pie with TOPPING american cheese with COMPLEX_TOPPING QUANTITY TOPPING oregano
(ORDER (PIZZAORDER (SIZE party sized ) (STYLE high rise dough ) pie with (TOPPING american cheese ) and with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING oregano ) ) ) )

i need to order one large pizza with ham bacon onions and black olives one medium pizza with sausage and onions and six large cokes

i need to order one large pizza with ham bacon onions and  black olives one medium pizza with sausage  and  onions and six large cokes




In [12]:
max_str_1 = len(max(X_train, key=len))
max_str_2 = len(max(y_train, key=len))
max_str_1, max_str_2

(116, 267)

In [13]:
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}  # Special tokens

def tokenize_output(output):
    """
    Tokenizes the structured output into meaningful tokens.
    Example:
        Input: "(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )"
        Output: ["(ORDER", "(PIZZAORDER", "(NUMBER", "a", "(SIZE", "large", "(TOPPING", "bbq", "pulled", "pork", ")", ")", ")", ")"]
    """
    tokens = re.findall(r"\(|\)|\w+|[^\s()]+", output)
    return tokens

def build_vocab(outputs, index):
    """
    Builds a vocabulary from tokenized outputs.
    """
    i = index
    for output in outputs:
        tokens = tokenize_output(output)
        for token in tokens:
            if token not in vocab:
                vocab[token] = i
                i += 1
    return vocab, i
def encode_outputs(outputs, vocab):
    """
    Encodes tokenized outputs into sequences of integers.
    """
    encoded = []
    for output in outputs:
        tokens = tokenize_output(output)
        sequence =  [vocab[token] for token in tokens if token in vocab]#[vocab["<SOS>"]] + + [vocab["<EOS>"]]
        encoded.append(sequence)
    return encoded

def pad_sequences_to_fixed_length(sequences, max_len):
    """
    Pads sequences to a fixed length.
    """
    return pad_sequences(sequences, maxlen=max_len, padding="post", value=0)

def decode_sequence(sequence, vocab):
    """
    Decodes a sequence of integers back into the structured output string.
    """
    inv_vocab = {v: k for k, v in vocab.items()}  # Reverse the vocabulary
    tokens = [inv_vocab[idx] for idx in sequence if idx in inv_vocab and idx not in { vocab["<SOS>"], vocab["<EOS>"],vocab["<PAD>"]} ] 
    output = " ".join(tokens)
    output = output.replace(" ( ", " (").replace("( ", "(") #.replace(" )", ")")
    return output

def decode_sequence_2(sequence, vocab):
    """
    Decodes a sequence of integers back into the structured output string.
    """
    inv_vocab = {v: k for k, v in vocab.items()}  # Reverse the vocabulary
    # sequence = sequence.cpu().tolist()  # Convert tensor to a list of integers
    tokens = [inv_vocab.get(idx, "") for idx in sequence if idx > 0]  # Ignore unknown and put empty char
    return "".join(tokens)



In [14]:
# import gensim.downloader as api
# glove_vectors = api.load("glove-wiki-gigaword-100")  # 100-dimension GloVe
# glove_vectors

In [15]:


def prepare_data(
    X_train, y_train, X_test, y_test, feature_type="bow", glove_vectors=None, max_len_1=20, max_len_2 = 20
):

    vectorizer = None

    # # Feature Extraction for X_train and X_test
    # if feature_type == "bow":
    #     X_train_processed, vectorizer = bag_of_words(X_train)
    #     X_train_processed = X_train_processed.toarray()
    #     X_test_processed = vectorizer.transform(X_test).toarray()
    # elif feature_type == "tfidf":
    #     X_train_processed, vectorizer = tfidf_features(X_train)
    #     X_train_processed = X_train_processed.toarray()
    #     X_test_processed = vectorizer.transform(X_test).toarray()
    # elif feature_type == "embeddings":
    #     if not glove_vectors:
    #         raise ValueError("GloVe vectors must be provided for embeddings.")
    #     X_train_tokenized = [sentence.split() for sentence in X_train]
    #     X_test_tokenized = [sentence.split() for sentence in X_test]
    #     X_train_processed = extract_embeddings(X_train_tokenized)
    #     X_test_processed = extract_embeddings(X_test_tokenized)
    # else:
    #     raise ValueError("Invalid feature type. Choose 'bow', 'tfidf', or 'embeddings'.")
    index = 3
    X_vocab, index = build_vocab(X_train,index)  # Build vocabulary from training outputs
    X_train_encoded = encode_outputs(X_train, X_vocab)  # Encode training outputs
    X_test_encoded = encode_outputs(X_test, X_vocab)  # Encode testing outputs
    X_train_processed = pad_sequences_to_fixed_length(X_train_encoded, max_len_1)
    X_test_processed = pad_sequences_to_fixed_length(X_test_encoded, max_len_1)

    y_vocab, index = build_vocab(y_train,index)  # Build vocabulary from training outputs
    y_train_encoded = encode_outputs(y_train, vocab)  # Encode training outputs
    y_test_encoded = encode_outputs(y_test, vocab)  # Encode testing outputs
    y_train_processed = pad_sequences_to_fixed_length(y_train_encoded, max_len_2)
    y_test_processed = pad_sequences_to_fixed_length(y_test_encoded, max_len_2)


    return (
        X_train_processed,
        X_test_processed,
        y_train_processed,
        y_test_processed,
        X_vocab,
        y_vocab,  # Return vocabulary for decoding
    )


In [16]:
X_train_processed, X_test_processed, y_train_processed, y_test_processed, X_vocab, y_vocab = prepare_data( X_train, y_train, X_test, y_test, feature_type="embeddings", max_len_1=270, max_len_2=270)

In [17]:
del X_train
del X_test
del y_train
del y_test

In [18]:
X_vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 'a': 3,
 'large': 4,
 'bbq': 5,
 'pulled': 6,
 'pork': 7,
 'pie': 8,
 'with': 9,
 'green': 10,
 'pepper': 11,
 'extra': 12,
 'pepperoni': 13,
 'like': 14,
 'vegetarian': 15,
 'pizza': 16,
 'party': 17,
 'size': 18,
 'stuffed': 19,
 'crust': 20,
 'american': 21,
 'cheese': 22,
 'mushroom': 23,
 'one': 24,
 'personal': 25,
 'sized': 26,
 'artichoke': 27,
 'banana': 28,
 'low': 29,
 'fat': 30,
 'want': 31,
 'regular': 32,
 'without': 33,
 'any': 34,
 'fried': 35,
 'onion': 36,
 'little': 37,
 'bit': 38,
 'high': 39,
 'rise': 40,
 'dough': 41,
 'lot': 42,
 'olive': 43,
 'pesto': 44,
 'sauce': 45,
 'yellow': 46,
 'meatball': 47,
 'bean': 48,
 'big': 49,
 'meat': 50,
 'pecorino': 51,
 'balsamic': 52,
 'glaze': 53,
 'black': 54,
 'chicken': 55,
 'mozzarella': 56,
 'italian': 57,
 'sausage': 58,
 'lunch': 59,
 'alfredo': 60,
 'need': 61,
 'cheeseburger': 62,
 'combination': 63,
 'spiced': 64,
 'mediterranean': 65,
 'caramelized': 66,
 'red': 67,
 'no': 6

In [19]:
vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 'a': 3,
 'large': 4,
 'bbq': 5,
 'pulled': 6,
 'pork': 7,
 'pie': 8,
 'with': 9,
 'green': 10,
 'pepper': 11,
 'extra': 12,
 'pepperoni': 13,
 'like': 14,
 'vegetarian': 15,
 'pizza': 16,
 'party': 17,
 'size': 18,
 'stuffed': 19,
 'crust': 20,
 'american': 21,
 'cheese': 22,
 'mushroom': 23,
 'one': 24,
 'personal': 25,
 'sized': 26,
 'artichoke': 27,
 'banana': 28,
 'low': 29,
 'fat': 30,
 'want': 31,
 'regular': 32,
 'without': 33,
 'any': 34,
 'fried': 35,
 'onion': 36,
 'little': 37,
 'bit': 38,
 'high': 39,
 'rise': 40,
 'dough': 41,
 'lot': 42,
 'olive': 43,
 'pesto': 44,
 'sauce': 45,
 'yellow': 46,
 'meatball': 47,
 'bean': 48,
 'big': 49,
 'meat': 50,
 'pecorino': 51,
 'balsamic': 52,
 'glaze': 53,
 'black': 54,
 'chicken': 55,
 'mozzarella': 56,
 'italian': 57,
 'sausage': 58,
 'lunch': 59,
 'alfredo': 60,
 'need': 61,
 'cheeseburger': 62,
 'combination': 63,
 'spiced': 64,
 'mediterranean': 65,
 'caramelized': 66,
 'red': 67,
 'no': 6

In [20]:
X_train_processed.shape, y_train_processed.shape

((2456446, 270), (2456446, 270))

In [21]:
X_train_processed[1]

array([ 4,  8,  9, 10, 11,  9, 12, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [22]:
y_train_processed[1]

array([240, 241, 243,   4,   8,   9, 244,  10,  11,   9, 245, 246,  12,
       244,  13,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [23]:

ou= decode_sequence(y_train_processed[1],vocab)
ou

'ORDER PIZZAORDER SIZE large pie with TOPPING green pepper with COMPLEX_TOPPING QUANTITY extra TOPPING pepperoni'

In [24]:
y_train[1]

NameError: name 'y_train' is not defined

In [ ]:
ou == y_train[1]

True

### Ensure Decoding works right

In [ ]:
total_sequences = 0
correct_sequences = 0

for src,tgt in zip(y_test_processed, y_test):
    #print(src)

    predicted_sequences = decode_sequence(src, vocab) 
    #print(predicted_sequences)
    # print(predicted_sequences)
    # print(target_sequences)
    # Calculate sequence accuracy
    if predicted_sequences == tgt:
        # print(pred)
        # print(tgt)
        correct_sequences += 1
    total_sequences += 1
print(f"Correct {correct_sequences}, Total {total_sequences}")
sequence_accuracy = correct_sequences / total_sequences if total_sequences > 0 else 0
sequence_accuracy * 100

Correct 81, Total 348


23.275862068965516

In [26]:
len(vocab)

252

In [27]:
input_dim = X_train_processed.shape[1]
input_dim

270

In [28]:
output_dim = y_train_processed.shape[1] 
output_dim

270

In [29]:
len(vocab)

252

In [39]:
input_dim =  X_train_processed.shape[1]  # Vocabulary size
embedding_dim = len(vocab)  # Dimension of embedding vectors    vocabs  260,158       max seq lens  (99, 265)
hidden_dim = 256  # Hidden state size for LSTM          first was 256 for batch norm 150
output_dim = y_train_processed.shape[1]  # Number of output classes
num_layers = 2  # Number of BiLSTM layers
dropout = 0.2  # Dropout probability

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)


In [31]:
import torch
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.long) 
        self.targets = torch.tensor(targets, dtype=torch.long) 

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "src_input_ids": self.inputs[idx],
            "tgt_input_ids": self.targets[idx],
        }

train_dataset = SequenceDataset(X_train_processed, y_train_processed)
test_dataset = SequenceDataset(X_test_processed, y_test_processed)

batch_size = 64  # Adjust based GPU ;-;  memory
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [32]:
train_dataset.__getitem__(3)

{'src_input_ids': tensor([17, 18, 19, 20,  8,  9, 21, 22,  9, 23,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0

In [40]:
def evaluate_model_with_accuracy(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    total_tokens = 0
    correct_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src)
            output_dim = output.shape[-1]

            # Flatten outputs and targets
            output = output.view(-1, output_dim)  # Shape: (batch_size * seq_len, output_dim)
            tgt = tgt.view(-1)  # Shape: (batch_size * seq_len)

            # # Apply mask to remove padding tokens
            # mask = tgt != 0  # Mask to ignore padding indices
            # output = output[mask]  # Filter model outputs
            # tgt = tgt[mask]  # Filter targets

            # Compute loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

            # Calculate accuracy
            predictions = output.argmax(dim=1)  # Get the index of the max log-probability
            correct_tokens += (predictions == tgt).sum().item()
            total_tokens += tgt.size(0)

    accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0
    return epoch_loss / len(dataloader), accuracy


In [41]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()  # Use for multi-class classification ignore_index=0 for padding
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=1e-5)

for epoch in range(20):  # Number of epochs
    model.train()
    epoch_loss = 0
    total_batches = len(train_dataloader)
    progress_bar = tqdm(train_dataloader, desc="Training Progress", unit="batch", leave=True)

    for batch_idx, batch in enumerate(progress_bar):  # Assuming a DataLoader is used
        src = batch["src_input_ids"].to(device)  # Input tokens
        tgt = batch["tgt_input_ids"].to(device)  # Target tokens

        optimizer.zero_grad()
        output = model(src)  # Forward pass
        output_dim = output.shape[-1]

        # Flatten outputs and targets for loss computation
        output = output.view(-1, output_dim)  # Shape: (batch_size * seq_len, output_dim)
        tgt = tgt.view(-1)  # Shape: (batch_size * seq_len)

        # Apply mask to remove padding tokens
        # mask = tgt != 0  # Mask to ignore padding indices
        # output = output[mask]  # Filter model outputs
        # tgt = tgt[mask]  # Filter targets
        # print(output.shape)
        # print(tgt.shape)

        # Compute loss
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        avg_loss = epoch_loss / (batch_idx + 1)
        progress_bar.set_description(f"Training Progress: Batch {batch_idx + 1}/{total_batches}, Avg Loss: {avg_loss:.8f}")

    val_loss, accuracy = evaluate_model_with_accuracy(model, test_dataloader, criterion, device)
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_dataloader):.8f}, Val Loss: {val_loss:.8f}, Accuracy: {accuracy * 100:.4f}%")


Training Progress: Batch 1041/38382, Avg Loss: 2.55793343:   3%|▎         | 1041/38382 [01:50<1:06:09,  9.41batch/s]


KeyboardInterrupt: 

### Saving the model

In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print(f"Model loaded from {path}")
    return model

In [ ]:
save_model(model, "../weights/Bilstm.pt")

Model saved to ../weights/Bilstm.pt


In [ ]:
model = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)
model = load_model(model,"../weights/Bilstm.pt")

Model loaded from ../weights/Bilstm.pt


### Testing Real Output sequence

In [ ]:
test_dataset.__getitem__(1)["src_input_ids"]

tensor([169,  78,  16,   9, 111, 113,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [ ]:

with torch.no_grad():

    output = model( test_dataset.__getitem__(1)["src_input_ids"].to(device))
    predictions = output.argmax(dim=1)
predictions

tensor([240, 241, 242, 169, 243,  78,  16,   9, 244, 111, 244, 113,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [ ]:
tgt= test_dataset.__getitem__(1)["tgt_input_ids"].to(device)
tgt

tensor([240, 241, 242, 169, 243,  78,  16,   9, 244, 111, 244, 113,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [ ]:

sequence = predictions.cpu().tolist()
pred_sequence= decode_sequence(sequence,vocab)
pred_sequence

'ORDER PIZZAORDER NUMBER five SIZE medium pizza with TOPPING tomato TOPPING ham'

In [ ]:
sequence = tgt.cpu().tolist()
tgt_sequence= decode_sequence(sequence,vocab)
tgt_sequence

'ORDER PIZZAORDER NUMBER five SIZE medium pizza with TOPPING tomato TOPPING ham'

In [ ]:
pred_sequence == tgt_sequence

True

In [ ]:
def evaluate_model_with_sequence_accuracy(model, dataloader, device):
    model.eval()
    total_sequences = 0
    correct_sequences = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src)
            output = output.argmax(dim=-1)

            # Decode sequences for comparison

            predicted_sequences = [decode_sequence(seq.cpu().tolist(), vocab) for seq in output]
            target_sequences = [decode_sequence(seq.cpu().tolist(), vocab) for seq in tgt]
            # print(predicted_sequences)
            # print(target_sequences)
            # Calculate sequence accuracy
            for pred, tgt in zip(predicted_sequences, target_sequences):
                if pred == tgt:
                    # print(pred)
                    # print(tgt)
                    correct_sequences += 1
                else:
                    print(pred)
                    print(tgt)
                total_sequences += 1

    print(f"Correct {correct_sequences}, Total {total_sequences}")
    sequence_accuracy = correct_sequences / total_sequences if total_sequences > 0 else 0
    return sequence_accuracy * 100

In [ ]:
evaluate_model_with_sequence_accuracy(model, test_dataloader, device)

ORDER want PIZZAORDER NUMBER two SIZE medium pizza with TOPPING sausage TOPPING black olive PIZZAORDER NUMBER two SIZE medium pizza with COMPLEX_TOPPING QUANTITY extra TOPPING american cheese COMPLEX_TOPPING QUANTITY extra TOPPING TOPPING pineaple
ORDER want PIZZAORDER NUMBER two SIZE medium pizza with TOPPING sausage TOPPING black olive PIZZAORDER NUMBER two SIZE medium pizza with TOPPING pepperoni COMPLEX_TOPPING QUANTITY extra TOPPING cheese PIZZAORDER NUMBER three SIZE large pizza with TOPPING pepperoni TOPPING sausage
ORDER like PIZZAORDER NUMBER a SIZE large TOPPING onion pepper pizza
ORDER like PIZZAORDER NUMBER a SIZE large TOPPING onion TOPPING pepper pizza
ORDER PIZZAORDER NUMBER one pie with TOPPING pesto TOPPING ham no NOT TOPPING olive
ORDER PIZZAORDER NUMBER one pie with TOPPING pesto TOPPING ham avoid NOT TOPPING olive
ORDER need PIZZAORDER NUMBER one SIZE large pizza with TOPPING ham TOPPING bacon TOPPING white TOPPING olive olive PIZZAORDER NUMBER medium medium TOPPING

27.011494252873565